In [1]:
import os

import numpy as np
import importlib
import gymnasium as gym
import matplotlib.pyplot as plt
from matplotlib import patches
import matplotlib.animation as manimation
from sklearn.decomposition import PCA


import gym_env
import utils
from utils import create_mapping, get_transition_matrix, create_mapping_nb, get_full_maze_values

## Helper Functions

In [2]:
def update_terminal_reward(agent, loc, r):
    """
    Update the reward for the terminal state of the agent according to loc

    Args:
    agent (LinearRL class) : The LinearRL agent
    loc (int) : The location to change (1 or 2)
    r (float) : The new reward to change r[loc] to
    """
    # Get location of reward and change
    r_loc = np.argwhere(agent.terminals)[loc]
    agent.r[r_loc] = r
    # Update expr_t inside of the agent
    agent.expr_t = np.exp(agent.r[agent.terminals] / agent._lambda)

def policy_reval(agent, r_new):
    """
    The New environment is the same as the old one except we 
    
    Args:
    agent (LinearRL class) : The LinearRL agent
    r_new (array) : Updated reward

    Returns:
    V_new (array) : New value of each state
    """
    expr_new = np.exp(r_new[agent.terminals] / agent._lambda)
    Z_new = np.zeros(len(r_new))

    Z_new[~agent.terminals] = agent.DR[~agent.terminals][:,~agent.terminals] @ agent.P @ expr_new
    Z_new[agent.terminals] = expr_new
    V_new = np.round(np.log(Z_new), 2)

    return V_new

## LinearRL-TD Model

In [3]:
class LinearRL:
    def __init__(self, env_name, alpha=0.1, beta=1, gamma=0.904, _lambda=1.0, epsilon=0.4, num_steps=25000, policy="random", imp_samp=True):
        self.env = gym.make(env_name)
        self.start_loc = self.env.unwrapped.start_loc
        self.target_locs = self.env.unwrapped.target_locs
        self.maze = self.env.unwrapped.maze
        self.walls = self.env.unwrapped.get_walls()
        self.size = self.maze.size - len(self.walls)   # Size of the state space is the = size of maze - number of blocked states
        self.height, self.width = self.maze.shape
        # self.target_locs = [self.target_loc]

        # Create mapping and Transition matrix
        self.mapping = create_mapping_nb(self.maze, self.walls)
        self.reverse_mapping = {index: (i, j) for (i, j), index in self.mapping.items()}
        self.T = get_transition_matrix(self.env, self.mapping)
        

        # Get terminal states
        self.terminals = np.diag(self.T) == 1
        # Calculate P = T_{NT}
        self.P = self.T[~self.terminals][:,self.terminals]
        # Set reward
        self.reward_nt = -1   # Non-terminal state reward
        self.reward_t = -1    # Terminal state reward
        self.r = np.full(len(self.T), self.reward_nt)
        self.r[self.terminals] = self.reward_t
        self.expr_t = np.exp(self.r[self.terminals] / _lambda)
        # Precalculate exp(r) for use with LinearRL equations
        self.expr_nt = np.exp(self.reward_nt / _lambda)

        # Params
        self.alpha = alpha
        self.beta = beta
        self.gamma = self.expr_nt
        self._lambda = _lambda
        self.epsilon = epsilon
        self.num_steps = num_steps
        self.policy = policy
        self.imp_samp = imp_samp

        # Model
        self.DR = self.get_DR()
        self.Z = np.full(self.size, 0.01)

        self.V = np.zeros(self.size)
        self.one_hot = np.eye(self.size)

    def get_states(self):
        """
        Returns all non-blocked states as well as a mapping of each state (i,j) -> to an index (k)
        """
        states = []
        index_mapping = {}
        index = 0
        for i in range(len(self.maze)):
            for j in range(len(self.maze[i])):
                if self.maze[i][j] in ['0', 'S', 'G']:
                    states.append((i, j))
                    index_mapping[(i, j)] = index
                    index += 1

        return states, index_mapping

    def get_DR(self):
        """
        Returns the DR initialization based on what decision policy we are using, values are filled with 0.01 if using softmax to avoid div by zero
        """
        if self.policy == "random":
            DR = np.eye(self.size)
            DR[np.where(self.terminals)[0], np.where(self.terminals)[0]] = (1/(1-self.gamma))
        
        elif self.policy == "softmax":
            DR = np.full((self.size, self.size), 0.01)
            np.fill_diagonal(DR, 1)
            DR[np.where(self.terminals)[0], np.where(self.terminals)[0]] = (1/(1-self.gamma))

        return DR

    def update_V(self):
        self.Z[~self.terminals] = self.DR[~self.terminals][:,~self.terminals] @ self.P @ self.expr_t
        self.Z[self.terminals] = self.expr_t
        self.V = np.round(np.log(self.Z), 2)
    
    def importance_sampling(self, state, s_prob):
        """
        Performs importance sampling P(x'|x)/u(x'|x). P(.) is the default policy, u(.) us the decision policy
        """
        successor_states = self.env.unwrapped.get_successor_states(state)
        p = 1/len(successor_states)
        w = p/s_prob
                
        return w

    def select_action(self, state, beta=0.5, target_loc=None):
        """
        Action selection based on our policy
        Options are: [random, softmax, egreedy, test]
        """
        if self.policy == "random":
            return self.env.unwrapped.random_action()
        
        elif self.policy == "softmax":
            successor_states = self.env.unwrapped.get_successor_states(state)      # succesor_states = [(state, terminated), ...]
            action_probs = np.full(self.env.action_space.n, 0.0)

            v_sum = sum(
                        np.exp((np.log(self.Z[self.mapping[(s[0][0],s[0][1])]] + 1e-20)) / self.beta) for s in successor_states
                        )

            # if we don't have enough info, random action
            if v_sum == 0:
                return self.env.unwrapped.random_action() 

            for action in self.env.unwrapped.get_available_actions(state):
                direction = self.env.unwrapped._action_to_direction[action]
                new_state = state + direction
                
                action_probs[action] = np.exp((np.log(self.Z[self.mapping[(new_state[0], new_state[1])]] + 1e-20)) / self.beta ) / v_sum

            action = np.random.choice(self.env.action_space.n, p=action_probs)
            s_prob = action_probs[action]

            return action, s_prob
    
        elif self.policy == "egreedy":
            if np.random.uniform(low=0, high=1) < self.epsilon:
                return self.env.unwrapped.random_action()
            else:
                action_values = np.full(self.env.action_space.n, -np.inf)
                for action in self.env.unwrapped.get_available_actions(state):
                    direction = self.env.unwrapped._action_to_direction[action]
                    new_state = state + direction

                    if self.maze[new_state[0], new_state[1]] == "1":
                        continue

                    action_values[action] = round(np.log(self.Z[self.mapping[(new_state[0],new_state[1])]]), 2)

                return np.argmax(action_values)
            
        elif self.policy == "test":
            action_values = np.full(self.env.action_space.n, -np.inf)
            for action in self.env.unwrapped.get_available_actions(state):
                direction = self.env.unwrapped._action_to_direction[action]
                new_state = state + direction

                # Need this to make it work for now
                if np.array_equal(new_state, target_loc):
                    return action

                if self.maze[new_state[0], new_state[1]] == "1":
                    continue
                action_values[action] = round(np.log(self.Z[self.mapping[(new_state[0],new_state[1])]]), 2)

            return np.nanargmax(action_values)

    def get_D_inv(self):
        """
        Calculates the DR directly using matrix inversion, used for testing
        """
        I = np.eye(self.size)
        D_inv = np.linalg.inv(I-self.gamma*self.T)

        return D_inv

    def learn(self):
        """
        Agent explores the maze according to its decision policy and and updates its DR as it goes
        """
        print(f"Decision Policy: {self.policy}, Number of Iterations: {self.num_steps}, lr={self.alpha}, temperature={self.beta}, importance sampling={self.imp_samp}")
        self.env.reset()

        # D_inv_1 = self.get_D_inv()
        # D_inv_2 = np.linalg.inv(np.diag(np.exp(-self.r))-self.T)

        # Iterate through number of steps
        for i in range(self.num_steps):
            # Current state
            state = self.env.unwrapped.agent_loc
            state_idx = self.mapping[(state[0], state[1])]

            # Choose action
            if self.policy == "softmax":
                action, s_prob = self.select_action(state)
            else:
                action = self.select_action(state, self.policy)
        
            # Take action
            obs, _, done, _, _ = self.env.step(action)

            # Unpack observation to get new state
            next_state = obs["agent"]
            next_state_idx = self.mapping[(next_state[0], next_state[1])]

            # Importance sampling
            if self.policy == "softmax":
                w = self.importance_sampling(state, s_prob)
                w = 1 if np.isnan(w) or w == 0 else w
            else:
                w = 1
            
            ## Update default representation
            target = self.one_hot[state_idx] + self.gamma * self.DR[next_state_idx]
            # If we are using importance sampling
            if self.imp_samp:
                self.DR[state_idx] = (1 - self.alpha) * self.DR[state_idx] + self.alpha * target * w
            else:
                self.DR[state_idx] = (1 - self.alpha) * self.DR[state_idx] + self.alpha * target

            ## Update Z-Values
            self.Z = self.DR[:,~self.terminals] @ self.P @ self.expr_t

            if done:
                self.env.reset()
                continue
            
            # Update state
            state = next_state

        # Update DR at terminal state
        self.Z[self.terminals] = np.exp(self.r[self.terminals] / self._lambda)
        self.V = np.round(np.log(self.Z), 2)


## Train Agents

In [4]:
mazes = ["simple-5x5-2", "simple-7x7-2"]
maze_name = mazes[1]

### D_inv agent

In [11]:
# Agent to be used with D_inv
agent = LinearRL(env_name=maze_name, _lambda=1.0, alpha=0.001, beta=1.0, num_steps=500000, policy="softmax", imp_samp=True)
# Make the reward for the first terminal state higher than the second to bias the DR towards that terminal state
update_terminal_reward(agent, loc=0, r=2)

D_inv = np.linalg.inv(np.diag(np.exp(-agent.r / agent._lambda)) - agent.T)

agent.DR = D_inv
agent.update_V()
maze_values = get_full_maze_values(agent)
print(maze_values)

[[ -9.6   -7.95  -6.29  -4.63  -3.26  -2.29  -2.85]
 [-11.22   -inf   -inf  -3.24  -1.6   -0.32  -1.54]
 [-12.07   -inf   -inf   -inf   0.33   2.    -0.04]
 [-10.94  -9.72   -inf   -inf   -inf  -0.05  -1.43]
 [ -9.45  -8.08  -6.28  -4.96   -inf  -2.09  -3.09]
 [ -8.05  -6.61  -4.84  -3.32  -4.42  -4.27  -4.87]
 [ -6.75  -5.11  -3.06  -1.    -3.05  -4.84  -5.86]]


In [12]:
# Update the second terminal state to make it double that of the first
update_terminal_reward(agent, loc=1, r=6)
V_new = policy_reval(agent=agent, r_new=agent.r)

In [13]:
agent.V = V_new
maze_values = get_full_maze_values(agent)
print(maze_values)

[[-8.52 -7.88 -6.29 -4.63 -3.26 -2.29 -2.85]
 [-7.25  -inf  -inf -3.24 -1.6  -0.32 -1.54]
 [-5.61  -inf  -inf  -inf  0.33  2.   -0.04]
 [-3.96 -2.72  -inf  -inf  -inf -0.02 -1.37]
 [-2.45 -1.09  0.72  2.04  -inf -1.06 -2.11]
 [-1.05  0.39  2.16  3.68  2.43  0.55 -0.95]
 [ 0.25  1.89  3.94  6.    3.95  1.9   0.27]]


### Agent with importance sampling

In [22]:
# Initialize the agent
agent_with_imp = LinearRL(env_name="simple-7x7-2", _lambda=1.0, alpha=0.001, beta=0.5, num_steps=500000, policy="softmax", imp_samp=True)
update_terminal_reward(agent_with_imp, loc=0, r=2)


In [23]:
# Train the agent with importance sampling
agent_with_imp.learn()

Decision Policy: softmax, Number of Iterations: 500000, lr=0.001, temperature=0.5


In [24]:
# Print out the values to see what it learned
maze_values = get_full_maze_values(agent_with_imp)
print(maze_values)

[[-7.07 -6.28 -5.13 -3.65 -2.28 -1.32 -2.2 ]
 [-6.47  -inf  -inf -2.25 -0.66  0.62 -0.55]
 [-5.93  -inf  -inf  -inf  1.35  2.    0.96]
 [-5.29 -4.56  -inf  -inf  -inf  0.9  -0.44]
 [-4.51 -4.4  -3.53 -2.99  -inf -1.08 -2.19]
 [-4.01 -3.77 -3.22 -2.07 -2.57 -2.35 -2.28]
 [-3.45 -3.35 -1.88 -1.   -1.65 -2.38 -2.28]]


In [25]:
update_terminal_reward(agent_with_imp, loc=1, r=6)
V_new = policy_reval(agent=agent_with_imp, r_new=agent_with_imp.r)

In [26]:
agent_with_imp.V = V_new
maze_values = get_full_maze_values(agent_with_imp)
print(maze_values)

[[-3.6  -3.01 -2.32 -1.47 -0.37  0.23  1.28]
 [-2.91  -inf  -inf -0.8  -0.4   0.84  0.57]
 [-2.36  -inf  -inf  -inf  1.53  2.    1.18]
 [-1.68 -0.88  -inf  -inf  -inf  1.1   0.26]
 [-0.8  -0.18  1.41  2.82  -inf  0.02  1.28]
 [ 0.01  1.23  3.09  4.66  2.99  1.23  1.28]
 [ 1.13  2.84  4.97  6.    4.92  2.    1.29]]


### Agent without importance sampling

In [34]:
agent_no_imp = LinearRL(env_name="simple-7x7-2", _lambda=1.0, alpha=0.001, beta=0.5, num_steps=500000, policy="softmax", imp_samp=False)
update_terminal_reward(agent_no_imp, loc=0, r=2)

In [35]:
# Train agent without importance sampling
agent_no_imp.learn()

Decision Policy: softmax, Number of Iterations: 500000, lr=0.001, temperature=0.5


In [36]:
# Print out the values to see what it learned
maze_values = get_full_maze_values(agent_no_imp)
print(maze_values)

[[-4.34 -3.78 -3.24 -2.32 -1.06 -0.21 -2.15]
 [-3.92  -inf  -inf -0.97  0.31  0.82 -0.04]
 [-3.69  -inf  -inf  -inf  1.43  2.    1.04]
 [-3.49 -3.09  -inf  -inf  -inf  1.05  0.04]
 [-3.11 -3.11 -2.66 -2.45  -inf  0.04 -2.09]
 [-2.89 -2.81 -2.55 -1.86 -2.33 -2.15 -2.23]
 [-2.6  -2.54 -1.73 -1.   -1.54 -2.26 -2.24]]


In [37]:
update_terminal_reward(agent_no_imp, loc=1, r=6)
V_new = policy_reval(agent=agent_no_imp, r_new=agent_no_imp.r)

In [38]:
agent_no_imp.V = V_new
maze_values = get_full_maze_values(agent_no_imp)
print(maze_values)

[[-0.79 -0.25  0.11  0.37  0.39 -0.17  1.3 ]
 [-0.36  -inf  -inf  0.4   0.32  0.82 -0.04]
 [-0.13  -inf  -inf  -inf  1.44  2.    1.04]
 [ 0.07  0.48  -inf  -inf  -inf  1.05  0.05]
 [ 0.46  0.54  1.39  2.71  -inf  0.05  1.28]
 [ 0.74  1.46  3.26  4.65  2.68  1.28  1.33]
 [ 1.37  3.3   4.93  6.    4.92  2.02  1.33]]
